# Visualization PNG Upload

This notebook uploads the visualizations in the `visualizations/` folder created in `lightning_threshold_innercore.ipynb` and `lightning_threshold_rainband.ipynb` to Google Drive. This notebook should be run after the creation of the visualizations in the previously mentioned notebooks. We use the [Google Drive API](https://developers.google.com/drive/api/guides/about-sdk) to facilitate interactions with Google Drive and use multithreading to speed up the uploading process.

### Code Requirements
This notebook requires following libraries not built in to Python:
- pydrive

### Setting Up Google Drive API
This code requires setting up a Google Cloud project and creating OAuth 2.0 Client ID.
 
1. Start by creating a Google Cloud project. We use the free tier. [Official Documentation](https://console.cloud.google.com/projectcreate?sjid=1200308559223672915-NC&inv=1&invt=AbjYiw)
2. Enable the Google Drive API. Navigate to the APIs & Services page, click the "Enable APIs and Services" button, and search for "Google Drive API". Click "Enable" if not already enabled.
3. Configure the OAuth consent screen as needed by navigating to APIs & Services > OAuth consent screen. For the sake of this project, we leave the app in testing mode and manually add test users for access. Users can be added in the "Test users" section. [Official Documentation](https://support.google.com/cloud/answer/10311615?hl=en&ref_topic=3473162&sjid=1200308559223672915-NC)
4. Create credentials. Navigate to APIs & Services > Credentials. Click "Create Credentials" and select "OAuth 2.0 Client ID". Select "Desktop app" as the application type and download the json file. Rename the file to `client_secret.json` and move this to the same directory as this script. [Official Documentation](https://support.google.com/cloud/answer/6158849?hl=en)

### Code

Start by importing the necessary libraries.

In [1]:
# Import packages
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import os
from threading import Thread
from queue import Queue

Next, authenticate with Google Drive. This will open your browser and prompt you to log in and connect to the app. Make sure the login used is included in the list of test users.

In [2]:
# Authenticate Google Drive
gauth = GoogleAuth()
gauth.LocalWebserverAuth()  # Authenticate via browser
drive = GoogleDrive(gauth)

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?client_id=389849867563-4uggnm57nqe52156v32gj1lkosoqpoem.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8080%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&access_type=offline&response_type=code

Authentication successful.


We set our parameters next, namely the destination folder ID. Make sure to have the destination folder created in Google Drive before performing this step.

The destination folder ID can be found as the string after the "folders/" part of the URL for the Google Drive folder.

In [3]:
# Folder ID of the destination folder in Google Drive
destination_folder_id = '1iIZx4ThnT8KyQc6pDPAemjYb0Ma_OFd_'  # Replace with folder ID
# Define parameters
file_extension = ".png"

Create a function to handle uploading the files to Google Drive.

In [4]:
# Function to upload files
def upload_file(printout=False):
    while not file_queue.empty():
        file_path = file_queue.get()
        try:
            file_name = os.path.basename(file_path)
            if printout:
                print(f"Uploading: {file_name}")
            gfile = drive.CreateFile({'title': file_name, 'parents': [{'id': destination_folder_id}]})
            gfile.SetContentFile(file_path)
            gfile.Upload()
            if printout:
                print(f"Uploaded: {file_name}")
        except Exception as e:
            print(f"Failed to upload {file_path}: {e}")
        file_queue.task_done()

Let's upload our files!

This code will look for `.png` files in the `visualizations/` directory to upload. We then upload these files concurrently over multiple threads that call the upload file function.

In [5]:
# Define the file upload queue
file_queue = Queue()

# Populate the queue with files to upload
base_directory = "visualizations/"
if not os.path.exists(base_directory):
    print(f"The path {base_directory} does not exist.")
    pass
else:
    for root, _, files in os.walk(base_directory):
        for file in files:
            if file.endswith(file_extension):
                file_path = os.path.join(root, file)
                file_queue.put(file_path)  # Add the file path to the queue

    # Check if files were added to the queue
    if file_queue.empty():
        print("No files were added to the queue.")
    else:
        total_queue = file_queue.qsize()
        print(f"Total files to upload: {total_queue}")

        # Create threads for uploading
        threads = []
        num_threads = 4

        for _ in range(num_threads):
            thread = Thread(target=upload_file)
            threads.append(thread)
            thread.start()

        # Wait for all threads to complete
        for thread in threads:
            thread.join()

        print(f"Uploaded files")

Total files to upload: 2172
Failed to upload visualizations/EPAC_17_16_Max_innercore_i3.png: EOF occurred in violation of protocol (_ssl.c:2427)
Uploaded files
